In [1]:
## import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, CuDNNLSTM, Activation, Bidirectional
import tensorflow as tf

Using TensorFlow backend.
/home/lab606a/.virtualenvs/keras/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/lab606a/.virtualenvs/keras/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/lab606a/.virtualenvs/keras/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/lab606a/.virtualenvs

In [2]:
# set GPU memory
gpu_options = tf.GPUOptions(allow_growth=True)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

In [7]:
datasetset = pd.read_csv("./datasets/split by random for distributed/classification/increasing_fixed_dataset.csv", header=None)
datasetset = datasetset.fillna(0)
datasetset = np.array(datasetset)
datasetset.shape

(104000, 91)

In [8]:
x_train = datasetset[:,1:]
x_train.shape

(104000, 90)

In [9]:
row = int(x_train.shape[1]/3)
col = 3

In [10]:
x_train = x_train.reshape(x_train.shape[0], row, col)
x_train = x_train.astype('float32')
x_train.shape

(104000, 30, 3)

In [11]:
y_train = datasetset[:,0]
y_train = y_train.astype('int')
y_train.shape

(104000,)

In [12]:
n_classes = 8

In [13]:
y_train = np_utils.to_categorical(y_train, n_classes)
y_train.shape

(104000, 8)

In [18]:
model = Sequential()
model.add(CuDNNLSTM(units=256, input_shape=(row,col), return_sequences=True))
model.add(CuDNNLSTM(units=128, return_sequences=True))
model.add(CuDNNLSTM(units=64, return_sequences=True))
model.add(CuDNNLSTM(units=32, return_sequences=True))
model.add(CuDNNLSTM(units=16, return_sequences=True))
model.add(CuDNNLSTM(units=8))
model.add(Dense(units=n_classes, activation='softmax'))

In [19]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_7 (CuDNNLSTM)     (None, 30, 256)           267264    
_________________________________________________________________
cu_dnnlstm_8 (CuDNNLSTM)     (None, 30, 128)           197632    
_________________________________________________________________
cu_dnnlstm_9 (CuDNNLSTM)     (None, 30, 64)            49664     
_________________________________________________________________
cu_dnnlstm_10 (CuDNNLSTM)    (None, 30, 32)            12544     
_________________________________________________________________
cu_dnnlstm_11 (CuDNNLSTM)    (None, 30, 16)            3200      
_________________________________________________________________
cu_dnnlstm_12 (CuDNNLSTM)    (None, 8)                 832       
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 72        
Total para

In [20]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [28]:
# training
batch_size = 8000
training_iters = 100
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=training_iters, shuffle=True)

Epoch 1/100
104000/104000 [==============================] - 3s 25us/step - loss: 0.0843 - acc: 0.9793
Epoch 2/100
104000/104000 [==============================] - 3s 25us/step - loss: 0.0879 - acc: 0.9772
Epoch 3/100
104000/104000 [==============================] - 3s 25us/step - loss: 0.0867 - acc: 0.9763
Epoch 4/100
104000/104000 [==============================] - 3s 25us/step - loss: 0.0870 - acc: 0.9763
Epoch 5/100
104000/104000 [==============================] - 3s 25us/step - loss: 0.0728 - acc: 0.9814
Epoch 6/100
104000/104000 [==============================] - 3s 25us/step - loss: 0.0722 - acc: 0.9812
Epoch 7/100
104000/104000 [==============================] - 3s 25us/step - loss: 0.0673 - acc: 0.9827
Epoch 8/100
104000/104000 [==============================] - 3s 25us/step - loss: 0.0706 - acc: 0.9807
Epoch 9/100
104000/104000 [==============================] - 3s 25us/step - loss: 0.0639 - acc: 0.9830
Epoch 10/100
104000/104000 [==============================] - 3s 25us/ste

104000/104000 [==============================] - 3s 25us/step - loss: 0.0881 - acc: 0.9754
Epoch 80/100
104000/104000 [==============================] - 3s 26us/step - loss: 0.0812 - acc: 0.9781
Epoch 81/100
104000/104000 [==============================] - 3s 25us/step - loss: 0.0764 - acc: 0.9790
Epoch 82/100
104000/104000 [==============================] - 3s 26us/step - loss: 0.0709 - acc: 0.9806
Epoch 83/100
104000/104000 [==============================] - 3s 26us/step - loss: 0.0685 - acc: 0.9811
Epoch 84/100
104000/104000 [==============================] - 3s 26us/step - loss: 0.0646 - acc: 0.9821
Epoch 85/100
104000/104000 [==============================] - 3s 25us/step - loss: 0.0615 - acc: 0.9833
Epoch 86/100
104000/104000 [==============================] - 3s 26us/step - loss: 0.0613 - acc: 0.9832
Epoch 87/100
104000/104000 [==============================] - 3s 25us/step - loss: 0.0587 - acc: 0.9837
Epoch 88/100
104000/104000 [==============================] - 3s 26us/step - 

In [22]:
def cal_acc(direction, speed):
    speed = str(speed)
    file = './datasets/split by random for distributed//classification/' +  direction + speed + '_test.csv'
    test_up2 = pd.read_csv(file, header=None)
    test_up2 = test_up2.fillna(0)
    test_up2 = np.array(test_up2)
    test_up2 = test_up2.reshape(test_up2.shape[0],row,col)
    cnt = np.array([0,0,0,0,0,0,0,0])
    pred = model.predict(test_up2)
    for i in range (pred.shape[0]):
        for j in range (8):
            if max(pred[i,:]) == pred[i,j]:
                cnt[j] += 1
    return cnt

In [29]:
accuarcy = np.zeros([8,8])
accuarcy[0,:] = cal_acc('top', 5)
accuarcy[1,:] = cal_acc('top', 6)
accuarcy[2,:] = cal_acc('left', 5)
accuarcy[3,:] = cal_acc('left', 6)
accuarcy[4,:] = cal_acc('right', 5)
accuarcy[5,:] = cal_acc('right', 6)
accuarcy[6,:] = cal_acc('back', 5)
accuarcy[7,:] = cal_acc('back', 6)
accuarcy

array([[242.,   8.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  5., 242.,   0.,   0.,   0.,   2.,   0.,   1.],
       [  1.,   0., 248.,   1.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   1., 248.,   0.,   0.,   1.,   0.],
       [  0.,   0.,   0.,   0., 249.,   1.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   1., 249.,   0.,   0.],
       [  0.,   0.,   2.,   3.,   0.,   0., 245.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   1.,   0., 249.]])

In [24]:
top5_test = pd.read_csv('./datasets/split by random for distributed/classification/top5_test.csv', header=None)
top5_test = np.array(top5_test)
top5_test.shape

(250, 90)

In [25]:
np.set_printoptions(suppress=True)
for i in range(14,90,3):
    tmp = top5_test[1,:i]
    tmp = tmp.reshape(1, tmp.shape[0])
    tmp = sequence.pad_sequences(tmp, maxlen=(row*col), padding='post', dtype='float32')
    tmp = tmp.reshape(1, 30, 3)
    print((i+1)/3)
    preddd = model.predict(tmp, verbose=1)
    print(preddd)

5.0
1/1 [==============================] - 0s 16ms/step
[[0.00000473 0.00138889 0.00030746 0.00170388 0.000074   0.0072361
  0.00005108 0.98923385]]
6.0
1/1 [==============================] - 0s 3ms/step
[[0.00000287 0.00003515 0.00000759 0.00022662 0.00736504 0.990824
  0.0007709  0.00076775]]
7.0
1/1 [==============================] - 0s 3ms/step
[[0.00011106 0.00134138 0.00003557 0.00007222 0.4317555  0.548493
  0.01776781 0.00042344]]
8.0
1/1 [==============================] - 0s 3ms/step
[[0.46385917 0.00224992 0.00185253 0.00025418 0.03963166 0.00062429
  0.49151167 0.00001657]]
9.0
1/1 [==============================] - 0s 3ms/step
[[0.98492485 0.00026195 0.00574772 0.00009056 0.00069688 0.00000312
  0.00827379 0.00000112]]
10.0
1/1 [==============================] - 0s 3ms/step
[[0.9827691  0.000274   0.0063886  0.00010529 0.00079356 0.00000383
  0.00966428 0.00000135]]
11.0
1/1 [==============================] - 0s 2ms/step
[[0.9753475  0.00037235 0.00902821 0.00016068 0.00132

In [34]:
#model.save('./classification_notfixed_20200505_256to8')